In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Step 3. Assemble DataFrame

This notebook demonstrates how harmonic features and ancillary features (like weather) are merged to produce a complete dataframe, which is then used to train a random forest in Google Earth Engine. The example data comes from the year 2018, in one of the grid cells generated in the previous R markdown file.

Steps:
1. Load harmonics and drop NaN (harmonics, cropland-CDL)
2. Load ancillary weather features and drop NaN
3. Merge harmonics and ancillary features
4. Drop non-crop points
5. Add target column (CDL) and useful identifiers
6. Keep only most important features
7. Save dataframe

In [2]:
# change these to the directory containing your data and the name of your data file
data_dir = '../data/example_samples/'
harmonics_file_name = 'harmonics_2terms_omega1-5_grid250pts_2018_0.csv'
ancillary_file_name = 'weatherVars_grid250pts_2018_0.csv'

### 1. Load harmonic coefficients

The harmonic regressions were of second order, so there are $5*6=30$ harmonic coefficients from raw Landsat bands and $5*13=65$ coefficients from vegetation indices. The vegetation indices considered were CRC, EVI, GCVI, GNDVI, NBR1, NBR2, NDTI, NDVI, NIRv, SNDVI, STI, TVI, and WDRVI. Two additional features considered are the GCVI max and date of GCVI max. Other columns serve to identify each sample uniquely.

In [3]:
harmonics = pd.read_csv(os.path.join(data_dir, harmonics_file_name))
harmonics = harmonics.drop(['system:index', '.geo'],axis=1) # drop extraneous columns from GEE export
print(harmonics.shape)
harmonics.head()

(250, 106)


,BLUE_constant,BLUE_cos,BLUE_cos2,BLUE_sin,BLUE_sin2,CRC_constant,CRC_cos,CRC_cos2,CRC_sin,CRC_sin2,...,WDRVI_sin2,confidence,cropland,fips5,gridID,lat,lon,state,uniqueID,year
0,0.035746,-0.009647,0.006359,-0.000103,-0.008163,0.488438,-0.026546,-0.023640,0.009429,-0.056738,...,0.085927,69,190,26001,195.0,44.831768,-83.572724,MI,0195_183,2018
1,0.027780,-0.005801,0.007712,-0.003289,-0.002800,0.529523,-0.001315,-0.001877,-0.050299,0.007275,...,0.022680,83,141,26007,195.0,44.867980,-83.572397,MI,0195_003,2018
2,0.030078,-0.004673,-0.000522,0.006260,-0.009113,0.414124,0.023964,-0.014955,-0.064167,0.052321,...,0.028260,100,190,26007,195.0,44.899742,-83.572895,MI,0195_235,2018
3,0.057390,-0.009631,0.009011,0.004697,-0.007497,0.497059,0.067664,0.012122,-0.010447,0.022476,...,0.305337,95,24,26007,195.0,44.955885,-83.543265,MI,0195_043,2018
4,0.081036,-0.022527,-0.006928,-0.001773,-0.003346,0.477893,0.020482,0.018140,0.001049,0.003594,...,-0.017057,73,5,26007,195.0,44.960926,-83.548727,MI,0195_015,2018


Create a clean version of the dataframe by dropping NaNs in the cropland and harmonics columns. Note that NaNs are due to:
- cropland column (for years/state where we don't have CDL)
- confidence (for CDL years < 2008 we don't have this information)
- harmonics not computable if there are too many cloudy Landsat images in a year

In [4]:
harmonics_clean = harmonics.dropna(subset = ['cropland', 'BLUE_constant'])
print(harmonics_clean.shape)

(243, 106)


### 2. Load ancillary features

In [5]:
ancillary = pd.read_csv(os.path.join(data_dir, ancillary_file_name))
ancillary = ancillary.drop(['system:index','.geo'],axis=1)
print(ancillary.shape)
ancillary.head()

(250, 26)


,GDD_ss_2018,aridity_2018,fips5,gridID,lat,lon,ppt_aug_2018,ppt_jul_2018,ppt_jun_2018,pr_early_2018,...,tmax_jul_2018,tmax_jun_2018,tmax_may_2018,tmin_aug_2018,tmin_jul_2018,tmin_jun_2018,tmin_may_2018,uniqueID,vpd_july_2018,vpd_jun_2018
0,1066,0.599215,26001,195.0,44.831768,-83.572724,121.251869,102.682827,25.270888,264.973820,...,28.022028,23.638727,22.139154,15.028436,14.045862,9.873376,6.924554,0195_183,0.964539,0.732099
1,1102,0.586854,26007,195.0,44.867980,-83.572397,107.063371,121.908918,24.078293,263.616762,...,29.039758,24.061151,22.326837,15.110895,14.202173,9.797357,6.904840,0195_003,1.082327,0.766172
2,1106,0.536362,26007,195.0,44.899742,-83.572895,96.006312,111.887820,22.313484,258.008988,...,28.824579,23.583154,21.814600,15.423761,14.580286,10.121484,7.116907,0195_235,1.053352,0.722594
3,1076,0.457237,26007,195.0,44.955885,-83.543265,93.942330,75.806609,19.412630,226.917306,...,27.484644,22.154108,20.159845,16.190729,15.503778,11.040704,7.883661,0195_043,0.900602,0.609829
4,1082,0.454485,26007,195.0,44.960926,-83.548727,97.292018,71.525389,22.716142,228.330198,...,28.146478,22.883386,21.042566,15.689233,14.913477,10.294855,7.302911,0195_015,0.974917,0.661405


We're interested in 20 weather features. The other 6 columns serve to identify the sample uniquely.

In [6]:
admin = ['uniqueID', 'lat', 'lon', 'gridID', 'state', 'fips5']
wvars = ['GDD_ss','aridity', 'ppt_aug', 'ppt_jul', 'ppt_jun', 'pr_early', 
         'pr_grow', 'tc_def_jul', 'tc_def_may', 'tc_soilm_aug', 'tmax_aug', 
         'tmax_jul', 'tmax_jun', 'tmax_may', 'tmin_aug', 'tmin_jul', 'tmin_jun', 
         'tmin_may', 'vpd_july', 'vpd_jun']

In [7]:
ancillary_long = pd.wide_to_long(ancillary, stubnames=wvars, i=admin, j="year", sep='_').reset_index()
ancillary_long.head()

,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,tmax_aug,tmax_jul,tmax_jun,tmax_may,tmin_aug,tmin_jul,tmin_jun,tmin_may,vpd_july,vpd_jun
0,0195_183,44.831768,-83.572724,195.0,MI,26001,2018,1066,0.599215,121.251869,...,26.414026,28.022028,23.638727,22.139154,15.028436,14.045862,9.873376,6.924554,0.964539,0.732099
1,0195_003,44.867980,-83.572397,195.0,MI,26007,2018,1102,0.586854,107.063371,...,27.340662,29.039758,24.061151,22.326837,15.110895,14.202173,9.797357,6.904840,1.082327,0.766172
2,0195_235,44.899742,-83.572895,195.0,MI,26007,2018,1106,0.536362,96.006312,...,27.517358,28.824579,23.583154,21.814600,15.423761,14.580286,10.121484,7.116907,1.053352,0.722594
3,0195_043,44.955885,-83.543265,195.0,MI,26007,2018,1076,0.457237,93.942330,...,26.341669,27.484644,22.154108,20.159845,16.190729,15.503778,11.040704,7.883661,0.900602,0.609829
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2018,1082,0.454485,97.292018,...,26.811578,28.146478,22.883386,21.042566,15.689233,14.913477,10.294855,7.302911,0.974917,0.661405


Create clean weather dataframe with no missing data by dropping NaNs.

In [8]:
ancillary_clean = ancillary_long.dropna()
print(ancillary_clean.shape)

(250, 27)


### 3. Merge harmonics and ancillary data

In [9]:
cols = ['uniqueID', 'lat', 'lon', 'gridID', 'state', 'fips5', 'year']
df = ancillary_clean.merge(harmonics_clean, on=cols)
print(df.shape)
df.head()

(243, 126)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_cos2,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland
0,0195_183,44.831768,-83.572724,195.0,MI,26001,2018,1066,0.599215,121.251869,...,-2.339169,-3.379903,1.184477,0.014195,0.276488,-0.138294,-0.139965,0.085927,69,190
1,0195_003,44.867980,-83.572397,195.0,MI,26007,2018,1102,0.586854,107.063371,...,-2.855503,-6.567488,1.880932,0.167989,0.258529,-0.188651,-0.182481,0.022680,83,141
2,0195_235,44.899742,-83.572895,195.0,MI,26007,2018,1106,0.536362,96.006312,...,0.466928,-3.475926,-0.325240,0.140537,0.082991,-0.011772,-0.158973,0.028260,100,190
3,0195_043,44.955885,-83.543265,195.0,MI,26007,2018,1076,0.457237,93.942330,...,-2.473902,-4.275414,7.658629,-0.377031,0.089564,-0.106706,-0.130135,0.305337,95,24
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2018,1082,0.454485,97.292018,...,2.879598,-3.671775,-0.364623,-0.314090,0.286671,0.120973,-0.074029,-0.017057,73,5


### 4. Filter to crop points only

In [10]:
# subset df to only crop classes
df['cropbinary'] = (((df['cropland'] <= 60) | (df['cropland'] >= 196)) | 
                    ((df['cropland'] >= 66) & (df['cropland'] <= 77)))
df = df[df['cropbinary']==True]
df = df.drop('cropbinary', axis=1)
print(df.shape)
df.head()

(30, 126)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_cos2,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland
3,0195_043,44.955885,-83.543265,195.0,MI,26007,2018,1076,0.457237,93.942330,...,-2.473902,-4.275414,7.658629,-0.377031,0.089564,-0.106706,-0.130135,0.305337,95,24
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2018,1082,0.454485,97.292018,...,2.879598,-3.671775,-0.364623,-0.314090,0.286671,0.120973,-0.074029,-0.017057,73,5
6,0195_179,45.139288,-83.861948,195.0,MI,26007,2018,1099,0.456512,84.368364,...,-1.241343,-4.882452,8.634047,-0.188749,0.262620,-0.052588,-0.111647,0.320419,93,36
8,0195_060,45.166328,-83.760757,195.0,MI,26007,2018,1093,0.456551,86.575071,...,-0.420300,-6.457481,12.247928,0.005061,0.428114,-0.051164,-0.191777,0.582049,92,36
12,0195_170,45.151803,-83.602972,195.0,MI,26007,2018,1078,0.415694,82.570334,...,-1.592891,-4.222719,5.556712,-0.363900,0.192150,-0.041209,-0.123044,0.208691,89,24


### 5. Add CDL target column

Since we're creating a crop type map of corn, soybean, and other, we use the encoding `{0: other crops, 1: corn, 5: soy}` to match CDL in corn and soy.

In [11]:
df['CDL'] = 0
df.loc[df['cropland'] == 1, 'CDL'] = 1
df.loc[df['cropland'] == 5, 'CDL'] = 5
df.head()

,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,TVI_sin,TVI_sin2,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland,CDL
3,0195_043,44.955885,-83.543265,195.0,MI,26007,2018,1076,0.457237,93.942330,...,-4.275414,7.658629,-0.377031,0.089564,-0.106706,-0.130135,0.305337,95,24,0
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2018,1082,0.454485,97.292018,...,-3.671775,-0.364623,-0.314090,0.286671,0.120973,-0.074029,-0.017057,73,5,5
6,0195_179,45.139288,-83.861948,195.0,MI,26007,2018,1099,0.456512,84.368364,...,-4.882452,8.634047,-0.188749,0.262620,-0.052588,-0.111647,0.320419,93,36,0
8,0195_060,45.166328,-83.760757,195.0,MI,26007,2018,1093,0.456551,86.575071,...,-6.457481,12.247928,0.005061,0.428114,-0.051164,-0.191777,0.582049,92,36,0
12,0195_170,45.151803,-83.602972,195.0,MI,26007,2018,1078,0.415694,82.570334,...,-4.222719,5.556712,-0.363900,0.192150,-0.041209,-0.123044,0.208691,89,24,0


We also add `state_name` and `state_abbr` columns for convenience.

In [12]:
abbr_to_state = {'IL':'Illinois', 'IA':'Iowa', 'IN':'Indiana', 'NE':'Nebraska', 'ND':'North Dakota',
                 'SD':'South Dakota', 'MN':'Minnesota', 'WI':'Wisconsin', 'MI':'Michigan',
                 'KS':'Kansas','KY':'Kentucky', 'OH':'Ohio', 'MO':'Missouri'}

df['state_abbrs'] = df['state']
df['state_name'] = df['state'].replace(abbr_to_state)
print(df.shape)
df.head()

(30, 129)


,uniqueID,lat,lon,gridID,state,fips5,year,GDD_ss,aridity,ppt_aug,...,WDRVI_constant,WDRVI_cos,WDRVI_cos2,WDRVI_sin,WDRVI_sin2,confidence,cropland,CDL,state_abbrs,state_name
3,0195_043,44.955885,-83.543265,195.0,MI,26007,2018,1076,0.457237,93.942330,...,-0.377031,0.089564,-0.106706,-0.130135,0.305337,95,24,0,MI,Michigan
4,0195_015,44.960926,-83.548727,195.0,MI,26007,2018,1082,0.454485,97.292018,...,-0.314090,0.286671,0.120973,-0.074029,-0.017057,73,5,5,MI,Michigan
6,0195_179,45.139288,-83.861948,195.0,MI,26007,2018,1099,0.456512,84.368364,...,-0.188749,0.262620,-0.052588,-0.111647,0.320419,93,36,0,MI,Michigan
8,0195_060,45.166328,-83.760757,195.0,MI,26007,2018,1093,0.456551,86.575071,...,0.005061,0.428114,-0.051164,-0.191777,0.582049,92,36,0,MI,Michigan
12,0195_170,45.151803,-83.602972,195.0,MI,26007,2018,1078,0.415694,82.570334,...,-0.363900,0.192150,-0.041209,-0.123044,0.208691,89,24,0,MI,Michigan


### 6. Keep only most important features

We refer the reader to [Wang _et al_ (2019)](https://www.sciencedirect.com/science/article/pii/S0034425718305790) for a description of feature selection. Following the results from this paper and with some additional experimentation with weather features, we retain the GCVI, NIR, SWIR1, and SWIR2 bands only as inputs to the random forest.

In [13]:
def get_features(bands):
    
    feats = []
    suffixes = ['_constant', '_cos', '_sin','_cos2', '_sin2']
    for band in bands:
        for suffix in suffixes:
            feats.append(band + suffix)
    
    return feats

In [14]:
# 20 total features
admin = ['uniqueID', 'gridID', 'state', 'fips5', 'year', 'cropland']
extra = ['lat', 'lon', 'confidence']
target = ['CDL']
features = get_features(['GCVI', 'NIR', 'SWIR1', 'SWIR2'])
print(len(features))

20


In [15]:
df_final = df[admin+features+target] # subset column
print(df_final.shape)
df_final.head()

(30, 27)


,uniqueID,gridID,state,fips5,year,cropland,GCVI_constant,GCVI_cos,GCVI_sin,GCVI_cos2,...,SWIR1_cos,SWIR1_sin,SWIR1_cos2,SWIR1_sin2,SWIR2_constant,SWIR2_cos,SWIR2_sin,SWIR2_cos2,SWIR2_sin2,CDL
3,0195_043,195.0,MI,26007,2018,24,1.799879,0.451688,-0.822888,-0.673585,...,0.011447,-0.004848,0.038837,-0.016929,0.181226,-0.007301,0.002267,0.037281,-0.027359,0
4,0195_015,195.0,MI,26007,2018,5,2.035074,1.387487,-0.283933,0.611552,...,-0.051841,-0.026630,-0.016050,0.008343,0.225915,-0.067665,-0.023903,-0.025352,0.005742,5
6,0195_179,195.0,MI,26007,2018,36,2.521701,1.139237,-0.469210,-0.213107,...,-0.033796,-0.009901,0.003583,-0.034356,0.157857,-0.038932,0.003362,0.005937,-0.041055,0
8,0195_060,195.0,MI,26007,2018,36,4.031029,3.211384,-1.121349,-0.124685,...,-0.001365,-0.025062,0.016978,-0.068587,0.137610,-0.026845,-0.006149,0.011383,-0.071164,0
12,0195_170,195.0,MI,26007,2018,24,1.655261,0.860312,-0.573015,-0.246277,...,-0.032973,0.002620,0.009036,-0.029157,0.184084,-0.050394,0.014009,0.012497,-0.040458,0


### 7. Save dataframe

In [16]:
df_final.to_csv(os.path.join(data_dir, 'allFeatures_grid250pts_2018_0.csv'), index=False)